In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Define the LinkedIn job search URL for data-related jobs across India (including Bengaluru)
base_url = "https://www.linkedin.com/jobs/search/?keywords=Data&location=India&f_TPR=r2592000"

# Set headers to mimic a browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Function to extract experience from job description using regex
def extract_experience(description):
    match = re.search(r"(\d+)[+-]?\s?(?:years?|yrs?)", description, re.IGNORECASE)
    return match.group(0) if match else "Not Mentioned"

# Function to scrape jobs from a single page
def scrape_page(url):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"❌ Failed to fetch page {url}")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    jobs = soup.find_all("div", class_="base-search-card")  # Adjust if needed
    
    job_list = []
    for job in jobs:
        title = job.find("h3", class_="base-search-card__title").text.strip() if job.find("h3") else "N/A"
        company = job.find("h4", class_="base-search-card__subtitle").text.strip() if job.find("h4") else "N/A"
        location = job.find("span", class_="job-search-card__location").text.strip() if job.find("span") else "N/A"
        link = job.find("a", class_="base-card__full-link")["href"] if job.find("a") else "N/A"
        date_posted = job.find("time")["datetime"] if job.find("time") else "N/A"  # Extract job posting date
        
        # Scrape individual job pages for skills and experience level
        job_response = requests.get(link, headers=headers)
        job_soup = BeautifulSoup(job_response.text, "html.parser")
        
        # Extract experience level from job description
        experience = "Not Mentioned"
        job_description = job_soup.find("div", class_="show-more-less-html__markup")
        if job_description:
            experience = extract_experience(job_description.text)
        
        # Extract skills from job description
        skills = "Not Mentioned"
        if job_description:
            skill_words = ["SQL", "Python", "Excel", "Power BI", "Tableau", "Machine Learning", "Data Analysis", "Statistics", "R", "Big Data"]
            found_skills = [word for word in skill_words if word.lower() in job_description.text.lower()]
            skills = ", ".join(found_skills) if found_skills else "Not Mentioned"
        
        job_list.append([title, company, location, date_posted, link, experience, skills])
    
    return job_list

# Scrape the first page
all_jobs = scrape_page(base_url)
print(f"✅ Scraped {len(all_jobs)} jobs from Page 1")

# Pagination Handling: Try to find and scrape additional pages
for page in range(25, 1000, 25):  # LinkedIn paginates in steps of 25 jobs
    paginated_url = base_url + f"&start={page}"
    jobs_on_page = scrape_page(paginated_url)
    
    if not jobs_on_page:  # If no more jobs, break the loop
        break
    
    all_jobs.extend(jobs_on_page)
    print(f"✅ Scraped {len(jobs_on_page)} jobs from Page {page//25 + 1}")
    time.sleep(2)  # Sleep to avoid being blocked

# Save to CSV
df = pd.DataFrame(all_jobs, columns=["Job Title", "Company", "Location", "Date Posted", "Job Link", "Experience Level", "Skills Required"])
df.to_csv("linkedin_data_jobs_india2.csv", index=False)

print(f"🎯 Scraping completed! Total jobs scraped: {len(df)}. Data saved to 'linkedin_data_jobs_india2.csv'.")


✅ Scraped 60 jobs from Page 1
✅ Scraped 7 jobs from Page 2
✅ Scraped 60 jobs from Page 3
✅ Scraped 7 jobs from Page 4
✅ Scraped 60 jobs from Page 5
✅ Scraped 60 jobs from Page 6
✅ Scraped 60 jobs from Page 7
✅ Scraped 60 jobs from Page 8
✅ Scraped 7 jobs from Page 9
✅ Scraped 60 jobs from Page 10
✅ Scraped 60 jobs from Page 11
✅ Scraped 60 jobs from Page 12
✅ Scraped 7 jobs from Page 13
✅ Scraped 7 jobs from Page 14
✅ Scraped 7 jobs from Page 15
✅ Scraped 60 jobs from Page 16
✅ Scraped 60 jobs from Page 17
✅ Scraped 7 jobs from Page 18
✅ Scraped 7 jobs from Page 19
✅ Scraped 60 jobs from Page 20
✅ Scraped 60 jobs from Page 21
✅ Scraped 60 jobs from Page 22
✅ Scraped 7 jobs from Page 23
✅ Scraped 60 jobs from Page 24
✅ Scraped 7 jobs from Page 25
✅ Scraped 7 jobs from Page 26
✅ Scraped 7 jobs from Page 27
✅ Scraped 60 jobs from Page 28
✅ Scraped 7 jobs from Page 29
✅ Scraped 7 jobs from Page 30
✅ Scraped 60 jobs from Page 31
✅ Scraped 60 jobs from Page 32
✅ Scraped 60 jobs from Page 33
